# ADAPT-VQE variation

Original: https://www.nature.com/articles/s41467-019-10988-2

This is an adaptive circuit... https://pennylane.ai/qml/demos/tutorial_adaptive_circuits

Qiskit community tutorial: https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/chemistry/adaptive_VQE.ipynb

Deprecated qiskit AdaptVQE implementation: https://docs.quantum.ibm.com/api/qiskit/0.44/qiskit.algorithms.minimum_eigensolvers.AdaptVQE

Review: https://link.springer.com/article/10.1186/S41313-021-00032-6

In [ ]:
# Qiskit community tutorial, implementation using qiskit's AdaptVQE()

import numpy as np

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

from qiskit_algorithms.optimizers import SLSQP
from qiskit_algorithms.minimum_eigensolvers import AdaptVQE, VQE

from qiskit.primitives import Estimator


info = MoleculeInfo(["Li", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, 1.5)])

driver = PySCFDriver.from_molecule(info, basis="sto3g")
molecule = driver.run()

transformer = FreezeCoreTransformer()
molecule = transformer.transform(molecule)
hamiltonian = molecule.hamiltonian.second_q_op()
mapper = ParityMapper(num_particles=molecule.num_particles)
tapered_mapper = molecule.get_tapered_mapper(mapper)
qubit_op = tapered_mapper.map(hamiltonian)

init_state = HartreeFock(
            molecule.num_spatial_orbitals,
            molecule.num_particles,
            tapered_mapper,
        )

estimator = Estimator()

optimizer = SLSQP(maxiter=10000, ftol=1e-9)



vqe_ansatz = UCCSD(
    molecule.num_spatial_orbitals,
    molecule.num_particles,
    tapered_mapper,
    initial_state=init_state
)
vqe = VQE(estimator, vqe_ansatz, optimizer)
vqe.initial_point = [0] * vqe_ansatz.num_parameters
algo = GroundStateEigensolver(tapered_mapper, vqe)
result_vqe = algo.solve(molecule)
energy_vqe = result_vqe.eigenvalues[0]




adapt_vqe_ansatz = UCCSD(
    molecule.num_spatial_orbitals,
    molecule.num_particles,
    tapered_mapper,
    initial_state=init_state
)

adapt_vqe = AdaptVQE(VQE(estimator, adapt_vqe_ansatz, optimizer))
result_adapt_vqe = adapt_vqe.compute_minimum_eigenvalue(qubit_op)
energy_adapt_vqe = result_adapt_vqe.eigenvalue


from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["Ansatz","Energy (Hartree)","Gates"]

vqe_cirq = result_vqe.raw_result.optimal_circuit
table.add_row(['UCCSD', str(energy_vqe), vqe_cirq.count_ops()])

adapt_vqe_cirq = result_adapt_vqe.optimal_circuit
table.add_row(['ADAPT-VQE', str(result_adapt_vqe.eigenvalue), adapt_vqe_cirq.count_ops()])

print(table)

/tmp/ipykernel_1407726/3558489826.py:36: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


+-----------+---------------------+--------------------------------+
|   Ansatz  |   Energy (Hartree)  |             Gates              |
+-----------+---------------------+--------------------------------+
|   UCCSD   | -1.1001883328996638 | OrderedDict({'EvolvedOps': 1}) |
| ADAPT-VQE | -1.1001795367509293 | OrderedDict({'EvolvedOps': 1}) |
+-----------+---------------------+--------------------------------+


> Need to expand according to the docs on github